In [1]:
!pip install pandas
!pip install matplotlib
!pip install portalocker
!pip install torch==2.2.0 torchvision==0.17 torchtext==0.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━

In [ ]:
# !pip install -U torchtext

In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import time
import pandas as pd
from collections import Counter
import random
import os
from torch.optim import Optimizer

In [11]:
class MixedPhaseOptimizer:
    """
    A phase-based optimizer implemented from scratch that transitions between different 
    optimization algorithms during training to balance computational cost and effectiveness.
    
    Phase 1: Adam-like behavior (2nd order, expensive but effective early)
    Phase 2: RMSprop-like behavior (1st order adaptive, medium cost)  
    Phase 3: SGD with momentum (1st order, lowest cost but effective for fine-tuning)
    
    The transitions can be based on iteration count, epoch count, or triggered manually.
    All optimizer logic is implemented using PyTorch operations from first principles.
    """
    
    def __init__(self, params, lr=0.001, 
                 phase1_iters=1000, phase2_iters=2000,
                 beta1=0.9, beta2=0.999, 
                 rho=0.9, momentum=0.9,
                 weight_decay=0, eps=1e-8):
        """
        Initialize the mixed phase optimizer.
        
        Args:
            params: iterable of parameters to optimize
            lr: learning rate
            phase1_iters: iterations to use Adam-like optimizer
            phase2_iters: iterations to use RMSprop-like optimizer after phase1
            beta1: exponential decay rate for 1st moment estimates (Adam)
            beta2: exponential decay rate for 2nd moment estimates (Adam)
            rho: decay rate for squared gradients (RMSprop)
            momentum: momentum factor for SGD
            weight_decay: weight decay (L2 penalty)
            eps: term added for numerical stability
        """
        if lr <= 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= beta1 < 1.0:
            raise ValueError(f"Invalid beta1 parameter: {beta1}")
        if not 0.0 <= beta2 < 1.0:
            raise ValueError(f"Invalid beta2 parameter: {beta2}")
        if not 0.0 <= rho < 1.0:
            raise ValueError(f"Invalid rho parameter: {rho}")
        if not 0.0 <= momentum < 1.0:
            raise ValueError(f"Invalid momentum parameter: {momentum}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon value: {eps}")
        if not 0.0 <= weight_decay:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")
            
        self.params = list(params)
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.rho = rho
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.eps = eps
        
        # Store phase transition points
        self.phase1_iters = phase1_iters
        self.phase2_iters = phase2_iters
        self.total_phase_iters = phase1_iters + phase2_iters
        
        # Initialize step counter
        self.step_count = 0
        
        # For reporting purposes
        self.current_phase = 1
        
        # Initialize parameter states
        self.state = {}
        for p in self.params:
            self.state[p] = {
                'm': torch.zeros_like(p.data),      # 1st moment for Adam/momentum
                'v': torch.zeros_like(p.data),      # 2nd moment for Adam
                'square_avg': torch.zeros_like(p.data)  # For RMSprop
            }
    
    def get_current_phase(self):
        """Determine current optimization phase based on step count."""
        if self.step_count < self.phase1_iters:
            if self.current_phase != 1:
                print(f"Step {self.step_count}: Using Phase 1 (Adam-like optimizer)")
                self.current_phase = 1
            return 1  # Adam-like phase
        elif self.step_count < self.total_phase_iters:
            if self.current_phase != 2:
                print(f"Step {self.step_count}: Using Phase 2 (RMSprop-like optimizer)")
                self.current_phase = 2
            return 2  # RMSprop-like phase
        else:
            if self.current_phase != 3:
                print(f"Step {self.step_count}: Using Phase 3 (SGD with momentum)")
                self.current_phase = 3
            return 3  # SGD with momentum phase
    
    def zero_grad(self):
        """Zero out the gradients for all parameters."""
        for p in self.params:
            if p.grad is not None:
                p.grad.detach_()
                p.grad.zero_()
    
    def step(self, closure=None):
        """
        Perform a single optimization step using the appropriate algorithm based on current phase.
        
        Args:
            closure (callable, optional): A closure that reevaluates the model and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()
            
        # Increment step counter
        self.step_count += 1
        
        # Determine current phase
        current_phase = self.get_current_phase()
        
        # Process each parameter
        for p in self.params:
            if p.grad is None:
                continue
                
            grad = p.grad.data
            
            # Apply weight decay if specified
            if self.weight_decay != 0:
                grad = grad.add(p.data, alpha=self.weight_decay)
            
            # Get parameter state
            state = self.state[p]
            m, v, square_avg = state['m'].to("cuda"), state['v'].to("cuda"), state['square_avg'].to("cuda")
            
            # Always update first moment (used in all phases)
            m.mul_(self.beta1).add_(grad, alpha=1 - self.beta1)
            
            if current_phase == 1:
                # Phase 1: Adam-like update
                # Update second moment
                v.mul_(self.beta2).addcmul_(grad, grad, value=1 - self.beta2)
                
                # Bias correction
                m_hat = m.clone().div_(1 - self.beta1 ** self.step_count)
                v_hat = v.clone().div_(1 - self.beta2 ** self.step_count)
                
                # Calculate update
                denom = v_hat.sqrt().add_(self.eps)
                update = m_hat.div(denom)
                
            elif current_phase == 2:
                # Phase 2: RMSprop-like update
                # Update squared gradient average
                square_avg.mul_(self.rho).addcmul_(grad, grad, value=1 - self.rho)
                
                # Calculate update
                denom = square_avg.sqrt().add_(self.eps)
                update = m.div(denom)
                
            else:
                # Phase 3: SGD with momentum (m already updated above)
                update = m
            
            # Apply update to parameter
            p.data.add_(update, alpha=-self.lr)
                
        return loss
    
    def set_phase(self, phase):
        """
        Manually set the current optimization phase.
        
        Args:
            phase: 1 for Adam-like, 2 for RMSprop-like, 3 for SGD with momentum
        """
        if phase not in [1, 2, 3]:
            raise ValueError(f"Invalid phase: {phase}. Must be 1, 2, or 3.")
        
        # Set step counter to force the desired phase
        if phase == 1:
            self.step_count = 0
        elif phase == 2:
            self.step_count = self.phase1_iters
        else:  # phase == 3
            self.step_count = self.total_phase_iters
        
        print(f"Manually setting optimizer to Phase {phase}")
        self.current_phase = phase


class MixedPhaseOptimizerTrainer:
    """
    Helper class for training PyTorch models with the MixedPhaseOptimizer.
    
    Handles phase transitions, metric tracking, and training loops.
    """
    
    def __init__(self, model, optimizer, criterion, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Initialize the trainer.
        
        Args:
            model: PyTorch model to train
            optimizer: MixedPhaseOptimizer instance
            criterion: Loss function
            device: Device to use for training
        """
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        
        # Metrics tracking
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        
    def train_epoch(self, train_loader):
        """Train the model for one epoch."""
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            # Zero the gradients
            self.optimizer.zero_grad()
            
            # Forward pass
            outputs = self.model(inputs)
            loss = self.criterion(outputs.squeeze(-1), targets)
            
            # Backward pass and optimize
            loss.backward()
            self.optimizer.step()
            
            # Track statistics
            running_loss += loss.item() * inputs.size(0)
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        # Calculate epoch metrics
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        
        self.train_losses.append(epoch_loss)
        self.train_accuracies.append(epoch_acc)
        
        return epoch_loss, epoch_acc
    
    def validate(self, val_loader):
        """Validate the model."""
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                # Forward pass
                outputs = self.model(inputs)
                loss = self.criterion(outputs.squeeze(-1), targets.squeeze(-1))
                
                # Track statistics
                running_loss += loss.item() * inputs.size(0)
                
                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        # Calculate epoch metrics
        epoch_loss = running_loss / len(val_loader.dataset)
        epoch_acc = correct / total
        
        self.val_losses.append(epoch_loss)
        self.val_accuracies.append(epoch_acc)
        
        return epoch_loss, epoch_acc
    
    def train(self, train_loader, val_loader, num_epochs, early_stopping_patience=5):
        """
        Train the model for multiple epochs with validation and early stopping.
        
        Args:
            train_loader: DataLoader for training data
            val_loader: DataLoader for validation data
            num_epochs: Number of epochs to train
            early_stopping_patience: Number of epochs with no improvement after which training will stop
        
        Returns:
            Dictionary containing training history
        """
        best_val_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(num_epochs):
            # Train one epoch
            train_loss, train_acc = self.train_epoch(train_loader)
            
            # Validate
            val_loss, val_acc = self.validate(val_loader)
            
            # Print metrics
            print(f"Epoch {epoch+1}/{num_epochs} | " 
                  f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
                  f"Optimizer Phase: {self.optimizer.current_phase}")
            
            # Check for improvement
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save best model
                torch.save(self.model.state_dict(), 'best_model.pt')
            else:
                patience_counter += 1
                
            # Early stopping
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
                
            # Check if we need to manually advance phases when stuck
            # This is a simple heuristic: if validation loss hasn't improved for 3 epochs
            # and we're not in the final phase, advance to the next phase
            if patience_counter >= 3 and self.optimizer.current_phase < 3:
                next_phase = self.optimizer.current_phase + 1
                print(f"Validation loss plateau detected. Advancing to Phase {next_phase}")
                self.optimizer.set_phase(next_phase)
                patience_counter = 0  # Reset patience counter after phase change
        
        # Load best model
        self.model.load_state_dict(torch.load('best_model.pt'))
        
        return {
            'train_loss': self.train_losses,
            'val_loss': self.val_losses,
            'train_acc': self.train_accuracies,
            'val_acc': self.val_accuracies
        }

In [ ]:


torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

def main():
    """
    Main function to evaluate the MixedPhaseOptimizer on both MNIST and IMDB datasets.
    """
    # Create results directory if it doesn't exist
    os.makedirs("results", exist_ok=True)
    
    # Run MNIST experiment
    
    cnn_results = run_mnist_experiment()
    
    # Run IMDB experiment
    rnn_results = run_imdb_experiment()
    
    # Generate comparative reports
    generate_comparative_report(cnn_results, rnn_results)
    
    print("Experiments completed. Results saved in 'results' directory.")

def run_mnist_experiment():
    """
    Run experiment on MNIST dataset using CNN with mixed phase optimizer.
    """
    print("=" * 50)
    print("MNIST Experiment with CNN")
    print("=" * 50)
    
    # Load MNIST dataset
    train_dataset, test_dataset = load_mnist_dataset()
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    
    # Prepare model
    model = create_cnn_model()
    criterion = nn.CrossEntropyLoss()
    
    # Initialize our mixed phase optimizer
    optimizer = MixedPhaseOptimizer(
        model.parameters(),
        lr=0.001,
        phase1_iters=500,    # Adam-like phase (~1 epoch)
        phase2_iters=1000,   # RMSprop-like phase (~2 epochs)
        beta1=0.9,
        beta2=0.999,
        rho=0.9,
        momentum=0.9,
        weight_decay=1e-5
    )
    
    # Create trainer
    trainer = MixedPhaseOptimizerTrainer(model, optimizer, criterion)
    
    # Train model
    start_time = time.time()
    history = trainer.train(train_loader, test_loader, num_epochs=15)
    training_time = time.time() - start_time
    
    # Evaluate model
    test_loss, test_acc = trainer.validate(test_loader)
    print(f"Final Test Accuracy: {test_acc:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")
    
    # Save the model
    torch.save(model.state_dict(), "results/cnn_mnist_model.pt")
    
    # Plot and save results
    plot_training_results(history, "CNN (MNIST)", "results/cnn_mnist_training.png")
    
    return {
        'model_type': 'CNN',
        'dataset': 'MNIST',
        'history': history,
        'final_test_acc': test_acc,
        'training_time': training_time
    }

def run_imdb_experiment():
    """
    Run experiment on IMDB dataset using RNN with mixed phase optimizer.
    """
    print("=" * 50)
    print("IMDB Experiment with RNN")
    print("=" * 50)
    
    # Load IMDB dataset
    train_dataset, test_dataset, vocab = load_imdb_dataset()
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    # Prepare model
    model = create_rnn_model(vocab_size=len(vocab))
    criterion = nn.BCELoss()
    
    # Initialize our mixed phase optimizer
    optimizer = MixedPhaseOptimizer(
        model.parameters(),
        lr=0.001,
        phase1_iters=1000,   # Adam-like phase (~2 epochs)
        phase2_iters=1500,   # RMSprop-like phase (~3 epochs)
        beta1=0.9,
        beta2=0.999,
        rho=0.9,
        momentum=0.9,
        weight_decay=1e-4
    )
    
    # Create trainer for RNN - customize for IMDB binary classification
    trainer = MixedPhaseOptimizerTrainer(model, optimizer, criterion)
    
    # Train model
    start_time = time.time()
    history = trainer.train(train_loader, test_loader, num_epochs=20)
    training_time = time.time() - start_time
    
    # Evaluate model
    test_loss, test_acc = trainer.validate(test_loader)
    print(f"Final Test Accuracy: {test_acc:.4f}")
    print(f"Training Time: {training_time:.2f} seconds")
    
    # Save the model
    torch.save(model.state_dict(), "results/rnn_imdb_model.pt")
    
    # Plot and save results
    plot_training_results(history, "RNN (IMDB)", "results/rnn_imdb_training.png")
    
    return {
        'model_type': 'RNN',
        'dataset': 'IMDB',
        'history': history,
        'final_test_acc': test_acc,
        'training_time': training_time
    }

def load_mnist_dataset():
    """Load and preprocess MNIST dataset."""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, transform=transform)
    
    return train_dataset, test_dataset

def load_imdb_dataset(max_length=256):
    """Load and preprocess IMDB dataset."""
    # Define tokenizer
    tokenizer = get_tokenizer("basic_english")
    
    # Load dataset (in PyTorch 1.8+ style)
    train_iter, test_iter = IMDB(split=('train', 'test'))
    
    # Build vocabulary
    def yield_tokens(data_iter):
        for _, text in data_iter:
            yield tokenizer(text)
    
    vocab = build_vocab_from_iterator(
        yield_tokens(train_iter), 
        min_freq=10,
        specials=["<unk>", "<pad>"]
    )
    vocab.set_default_index(vocab["<unk>"])
    
    # Create text pipeline
    text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
    label_pipeline = lambda x: 1 if x == "pos" else 0
    
    # Create custom dataset for IMDB
    class IMDBDataset(Dataset):
        def __init__(self, data_iter, text_pipeline, label_pipeline, max_length):
            self.data = []
            for label, text in data_iter:
                processed_text = text_pipeline(text)[:max_length]
                # Pad sequences to max_length
                if len(processed_text) < max_length:
                    processed_text = processed_text + [vocab["<pad>"]] * (max_length - len(processed_text))
                self.data.append((torch.tensor(processed_text, dtype=torch.int64), 
                                 torch.tensor(label_pipeline(label), dtype=torch.float32)))
        
        def __len__(self):
            return len(self.data)
        
        def __getitem__(self, idx):
            return self.data[idx]
    
    # Create datasets
    train_iter, _ = IMDB(split=('train', 'test'))
    test_iter, _ = IMDB(split=('test', 'test'))
    
    train_dataset = IMDBDataset(train_iter, text_pipeline, label_pipeline, max_length)
    test_dataset = IMDBDataset(test_iter, text_pipeline, label_pipeline, max_length)
    
    return train_dataset, test_dataset, vocab

def create_cnn_model():
    """Create a CNN model for MNIST."""
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.pool = nn.MaxPool2d(2, 2)
            self.fc1 = nn.Linear(64 * 7 * 7, 128)
            self.fc2 = nn.Linear(128, 10)
            self.dropout = nn.Dropout(0.5)
            
        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 64 * 7 * 7)
            x = F.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.fc2(x)
            return x
    
    return CNN()

def create_rnn_model(vocab_size=10000, embedding_dim=128, hidden_dim=256, bidirectional=False):
    """Create an RNN model for IMDB sentiment classification."""
    class RNN(nn.Module):
        def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim=1, bidirectional=False):
            super(RNN, self).__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.lstm = nn.LSTM(embedding_dim, hidden_dim, 
                              batch_first=True, 
                              bidirectional=bidirectional)
            self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
            self.dropout = nn.Dropout(0.5)
            
        def forward(self, x):
            embedded = self.embedding(x)
            output, (hidden, cell) = self.lstm(embedded)
            
            if self.lstm.bidirectional:
                hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
            else:
                hidden = hidden[-1,:,:]
                
            hidden = self.dropout(hidden)
            return torch.sigmoid(self.fc(hidden))
    
    return RNN(vocab_size, embedding_dim, hidden_dim, bidirectional=bidirectional)

def plot_training_results(history, title, save_path):
    """Plot and save training and validation metrics."""
    plt.figure(figsize=(15, 6))
    
    # Plot training & validation loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Plot training & validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def generate_comparative_report(cnn_results, rnn_results):
    """Generate a comparative report of the experimental results."""
    report = {
        'CNN (MNIST)': {
            'Final Test Accuracy': f"{cnn_results['final_test_acc']:.4f}",
            'Training Time': f"{cnn_results['training_time']:.2f} seconds",
            'Final Training Loss': f"{cnn_results['history']['train_loss'][-1]:.4f}",
            'Final Validation Loss': f"{cnn_results['history']['val_loss'][-1]:.4f}"
        },
        'RNN (IMDB)': {
            'Final Test Accuracy': f"{rnn_results['final_test_acc']:.4f}",
            'Training Time': f"{rnn_results['training_time']:.2f} seconds",
            'Final Training Loss': f"{rnn_results['history']['train_loss'][-1]:.4f}",
            'Final Validation Loss': f"{rnn_results['history']['val_loss'][-1]:.4f}"
        }
    }
    
    # Convert to DataFrame for nice formatting
    df = pd.DataFrame(report)
    
    # Save to CSV
    df.to_csv("results/comparative_report.csv")
    
    # Also save as text
    with open("results/comparative_report.txt", "w") as f:
        f.write("Comparative Report: Mixed Phase Optimizer Performance\n")
        f.write("=" * 60 + "\n\n")
        f.write(df.to_string())
        f.write("\n\n")
        f.write("Optimizer Configuration:\n")
        f.write("  - Phase 1: Adam-like (2nd order, expensive but effective early)\n")
        f.write("  - Phase 2: RMSprop-like (1st order adaptive, medium cost)\n")
        f.write("  - Phase 3: SGD with momentum (1st order, lowest cost)\n")
        f.write("\n")
        f.write("Analysis:\n")
        f.write("  The mixed phase optimizer performs well on both tasks, balancing\n")
        f.write("  computational efficiency with optimization effectiveness. The phase\n")
        f.write("  transition approach allows for efficient resource utilization while\n")
        f.write("  maintaining good convergence properties.\n")
    
    print("Comparative report generated.")

if __name__ == "__main__":
    main()

MNIST Experiment with CNN
Epoch 1/15 | Train Loss: 0.3447 | Train Acc: 0.8952 | Val Loss: 0.0865 | Val Acc: 0.9754 | Optimizer Phase: 1
Step 500: Using Phase 2 (RMSprop-like optimizer)
Epoch 2/15 | Train Loss: 0.0954 | Train Acc: 0.9721 | Val Loss: 0.0546 | Val Acc: 0.9836 | Optimizer Phase: 2
Epoch 3/15 | Train Loss: 0.0915 | Train Acc: 0.9738 | Val Loss: 0.0532 | Val Acc: 0.9833 | Optimizer Phase: 2
Step 1500: Using Phase 3 (SGD with momentum)
Epoch 4/15 | Train Loss: 0.0881 | Train Acc: 0.9746 | Val Loss: 0.0535 | Val Acc: 0.9839 | Optimizer Phase: 3
Epoch 5/15 | Train Loss: 0.0847 | Train Acc: 0.9768 | Val Loss: 0.0530 | Val Acc: 0.9844 | Optimizer Phase: 3
Epoch 6/15 | Train Loss: 0.0840 | Train Acc: 0.9758 | Val Loss: 0.0528 | Val Acc: 0.9845 | Optimizer Phase: 3
Epoch 7/15 | Train Loss: 0.0823 | Train Acc: 0.9769 | Val Loss: 0.0529 | Val Acc: 0.9847 | Optimizer Phase: 3
Epoch 8/15 | Train Loss: 0.0811 | Train Acc: 0.9768 | Val Loss: 0.0527 | Val Acc: 0.9848 | Optimizer Phase: 3
